### Split data

In [7]:
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
import json
import pandas as pd

def get_quetions_groundtruth_correct_contexts(qar_dataset_path: str):
    """ This method will return question, ground_truths, and correct_retrieved_contexts """
    def get_context(input_text):
        lines = input_text.split('\n')
        for idx, text in enumerate(lines):
            if text.startswith('Context'):
                # The next line will be the context string
                if idx + 1 < len(lines):
                    return lines[idx + 1]
    df = pd.read_csv(qar_dataset_path)
    node_ids = []
    questions = []
    ground_truths = []
    correct_contexts = []

    for row_id, obj_str in enumerate(df['objs']):
        objs = json.loads(obj_str)
        if len(objs) == 0:
            print(f"[Invalid rqa pairs] No objs found at row id: {row_id} which should have {df['qar_num'][row_id]} questions")
        for obj in objs:
            node_ids.append(df['node_id'][row_id])
            questions.append(obj['Question'])
            ground_truths.append(obj['Answer'])
            correct_contexts.append([get_context(df['input_text'][row_id])])
            
    new_df = pd.DataFrame(
        {
            'node_ids': node_ids,
            'questions': questions,
            'ground_truths': ground_truths,
            'correct_contexts': correct_contexts
        }
    )
    return new_df

qar_path = os.path.abspath('../../.save/gpt-4o-batch-all-target_1_parser/question/gpt-4o-batch-all-target_extract_gpt-4o-QAExtractor-batch_pid_0.jsonl.csv')
df = get_quetions_groundtruth_correct_contexts(qar_path)

[Invalid rqa pairs] No objs found at row id: 432 which should have 1 questions


In [8]:
df

,node_ids,questions,ground_truths,correct_contexts
0,3d751495-ce66-4254-a79f-ed8d740b9424,What structural factor allows for easy water a...,Cracked cuticles allow for easy water absorpti...,[This research investigates the differences in...
1,3d751495-ce66-4254-a79f-ed8d740b9424,Why do intact cuticles remain impermeable to w...,Intact cuticles remain impermeable due to the ...,[This research investigates the differences in...
2,3d751495-ce66-4254-a79f-ed8d740b9424,How can the permeability of intact cuticles be...,Heat and alkaline treatments can temporarily r...,[This research investigates the differences in...
3,01a93867-b15d-40e1-b287-4aa503b1b7b1,Why has the production of healthy sweeteners l...,The production of healthy sweeteners like fruc...,[The production of healthy sweeteners like fru...
4,01a93867-b15d-40e1-b287-4aa503b1b7b1,What are the methods mentioned for producing f...,Fructose can be made using either multi-enzyma...,[The production of healthy sweeteners like fru...
...,...,...,...,...
1113,d9eee2b4-e127-4db9-9404-5e69298f79f5,What are the primary lifestyle factors contrib...,Obesity is commonly caused by excessive food i...,[Obesity is one of very first warning signs fo...
1114,b1323ffb-d79c-4c60-a643-cb2535568e55,How do sedentary lifestyles affect energy meta...,Sedentary lifestyles affect the homeostasis of...,"[In fact, it is usually the consequence of sed..."
1115,b6995542-18ee-41df-8135-082c97a7b1c9,What potential strategies are suggested by the...,The research suggests both indirect and direct...,[Although the manufacturing of living A. mucin...
1116,b9b37f4d-93f5-4186-985f-d9fcf0d3e8d3,How do mental and physical conditions impact t...,Mental and physical conditions of the host sev...,[Gut microbiota symbiotically reside in human ...


In [13]:
from component.io import load_nodes_jsonl
node_path = os.path.abspath('../../.save/gpt-4o-batch-all-target_1_parser/sub/gpt-4o-batch-all-target_1_parser_ManuallyHierarchicalNodeParser_7652_gpu_V100_nodeNum_50_pid_0.jsonl')
nodes = load_nodes_jsonl(node_path)
node_Id_2_node = {node.id_: node for node in nodes}

Loading gpt-4o-batch-all-target_1_parser_ManuallyHierarchicalNodeParser_7652_gpu_V100_nodeNum_50_pid_0.jsonl: 100%|██████████| 144M/144M [00:03<00:00, 43.3MB/s] 


In [16]:
df['levels'] = [node_Id_2_node[node_id].metadata['level'] for node_id in df['node_ids']]

In [17]:
df

,node_ids,questions,ground_truths,correct_contexts,levels
0,3d751495-ce66-4254-a79f-ed8d740b9424,What structural factor allows for easy water a...,Cracked cuticles allow for easy water absorpti...,[This research investigates the differences in...,document
1,3d751495-ce66-4254-a79f-ed8d740b9424,Why do intact cuticles remain impermeable to w...,Intact cuticles remain impermeable due to the ...,[This research investigates the differences in...,document
2,3d751495-ce66-4254-a79f-ed8d740b9424,How can the permeability of intact cuticles be...,Heat and alkaline treatments can temporarily r...,[This research investigates the differences in...,document
3,01a93867-b15d-40e1-b287-4aa503b1b7b1,Why has the production of healthy sweeteners l...,The production of healthy sweeteners like fruc...,[The production of healthy sweeteners like fru...,document
4,01a93867-b15d-40e1-b287-4aa503b1b7b1,What are the methods mentioned for producing f...,Fructose can be made using either multi-enzyma...,[The production of healthy sweeteners like fru...,document
...,...,...,...,...,...
1113,d9eee2b4-e127-4db9-9404-5e69298f79f5,What are the primary lifestyle factors contrib...,Obesity is commonly caused by excessive food i...,[Obesity is one of very first warning signs fo...,multi-sentences
1114,b1323ffb-d79c-4c60-a643-cb2535568e55,How do sedentary lifestyles affect energy meta...,Sedentary lifestyles affect the homeostasis of...,"[In fact, it is usually the consequence of sed...",multi-sentences
1115,b6995542-18ee-41df-8135-082c97a7b1c9,What potential strategies are suggested by the...,The research suggests both indirect and direct...,[Although the manufacturing of living A. mucin...,multi-sentences
1116,b9b37f4d-93f5-4186-985f-d9fcf0d3e8d3,How do mental and physical conditions impact t...,Mental and physical conditions of the host sev...,[Gut microbiota symbiotically reside in human ...,multi-sentences


In [18]:
df['levels'].value_counts()

levels
section            404
paragraph          290
multi-sentences    274
document           150
Name: count, dtype: int64